# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [10]:
# number of callbacks for black-sounding and white-sounding names
print(sum(data[data.race=='b'].call))
print(sum(data[data.race=='w'].call))

157.0
235.0


# Q1: What test is appropriate for this problem? Does CLT apply?

In [5]:
data.describe()

,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,occupbroad,workinschool,...,educreq,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind
count,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,...,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000
mean,3.618480,3.661396,7.842916,0.052772,0.411499,0.097125,0.448049,215.637782,3.481520,0.559548,...,0.106776,0.437166,0.072690,0.082957,0.030390,0.085010,0.213963,0.267762,0.154825,0.165092
std,0.714997,1.219126,5.044612,0.223601,0.492156,0.296159,0.497345,148.127551,2.038036,0.496492,...,0.308866,0.496083,0.259649,0.275854,0.171677,0.278932,0.410141,0.442847,0.361773,0.371308
min,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,7.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,5.000000,0.000000,0.000000,0.000000,0.000000,27.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,4.000000,6.000000,0.000000,0.000000,0.000000,0.000000,267.000000,4.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.000000,4.000000,9.000000,0.000000,1.000000,0.000000,1.000000,313.000000,6.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,4.000000,7.000000,44.000000,1.000000,1.000000,1.000000,1.000000,903.000000,6.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
data.isnull().sum()

id                       0
ad                       0
education                0
ofjobs                   0
yearsexp                 0
honors                   0
volunteer                0
military                 0
empholes                 0
occupspecific            0
occupbroad               0
workinschool             0
email                    0
computerskills           0
specialskills            0
firstname                0
sex                      0
race                     0
h                        0
l                        0
call                     0
city                     0
kind                     0
adid                     0
fracblack               86
fracwhite               86
lmedhhinc               86
fracdropout             86
fraccolp                86
linc                    86
                      ... 
parent_emp            3148
branch_sales          4262
branch_emp            4212
fed                   1768
fracblack_empzip      2952
fracwhite_empzip      2952
l

Some columns have null value. But, the columns that are mainly for used EDA are race and call. These are not null. The data is clean to start EDA. 

Sample size is 4,870. CLT can be applied for this data. 

Because we are comparing two groups, a two-sample t-test is useful.

In [12]:
# Filter callback data by race:
black_callbacks = data.call[data.race == 'b']
white_callbacks = data.call[data.race == 'w']

# Number of black applicants
n_black = len(data[data.race == 'b'])

# Number of callbacks for black-sounding names
n_black_calls = black_callbacks.sum()

print('Total number of black applicants:', n_black)
print('Callbacks for black applicants:', n_black_calls)

# Number of white applicants
n_white = len(data[data.race == 'w'])

# Number of callbacks for white-sounding names
n_white_calls = white_callbacks.sum()

print('Total number of white applicants:', n_white)
print('Callbacks for white applicants:', n_white_calls)

Total number of black applicants: 2435
Callbacks for black applicants: 157.0
Total number of white applicants: 2435
Callbacks for white applicants: 235.0


Number of black and white applicants are equal. Instructions talked about high/low quality resumes. I am going to assume that resume quality factor is equally distributed. Based on above stats, it's clear that black applicants received less number of callbacks. Black applicants received 40% of the callbacks compared to 60% received by white applicants.

In [31]:
# Applying two sample t-test
t = stats.ttest_ind(black_callbacks, white_callbacks)[0]
p = stats.ttest_ind(black_callbacks, white_callbacks)[1]
print('t-statistic: {0:.7f}'.format(t))
print('p-value: {0:.7f}'.format(p))

t-statistic: -4.1147053
p-value: 0.0000394


In [26]:
# Print the quantiles of the t-distribution corresponding to
# confidence level and degrees of freedom:
n = len(data)
print((stats.t.ppf(q=0.0005, df=n-1)), (stats.t.ppf(q=0.9995, df=n-1)))

-3.292526135914183 3.292526135914214


The t-statistic falls outside the t-distribution corresponding to a 99.9% confidence level and the p-value is well below the significance level of 0.05, so the null hypothesis can be rejected.

# Q2: What are the null and alternate hypotheses?

Null hypothesis is that there is no difference between the callbacks received by black applicants and white applicants. 

Alternate hypothesis is there *is* a difference in the callbacks received by black applicants and white applicants.

# Q3: Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [28]:
from scipy.stats import norm

# “Critical" values of z are associated with interesting central areas under the standard normal curve. 
# Calculate the critical value of z at 99.95% confidence interval
z_critical = norm.ppf(0.9995)
z_critical

3.2905267314919255

In [45]:
# Number of black applicants = n_black
# Number of callbacks for black-sounding names = n_black_calls
# Number of white applicants = n_white
# Number of callbacks for white-sounding names = n_white_calls

# % of black applicants that were called
percentage_black = (n_black_calls/n_black) * 100
# % of white applicants that were called
percentage_white = (n_white_calls/n_white) * 100

# Variance for percentage callback of black applicants
P1 = percentage_black / 100
variance_black = (P1*(1-P1)/n_black)

# Variance for percentage callback of white applicants
P2 = percentage_white / 100
variance_white = (P2*(1-P2)/n_white)

variance_black_white = variance_black + variance_white

# Standard deviation for variance_black_white
std_black_white = np.sqrt(variance_black_white)

# For 95% confidence level, calculate margin of error
moe = 1.96 * std_black_white
print('Margin of error: {0:.7f}'.format(moe))

# confidence interval
CI = abs(P1-P2) + np.array([-1, 1]) * moe
print('Confidence interval', CI)

# p-value was calculated for previous question
print('p-value: {0:.7f}'.format(p))

Margin of error: 0.0152554
Confidence interval [0.01677745 0.04728826]
p-value: 0.0000394


# Q4: Write a story describing the statistical significance in the context of the original problem.

Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers. By applying various exploratory data analysis techniques, we found that applicants with white-sounding names are more likely receive the callback. 

This analysis proves that racial discrimination very much exists in U.S. labor market. 

# Q5: Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

Assuming that high quality and low quality resumes are equally distributed among black and white applicants, my analysis confirms that race is the important factor in callback success. 

There are few ways to explore this even further. We can explore if gender or gender + race played the role in callback success. Also, if we get more data about the kind of job (executive job, software developer, blue collar, sales, etc.,) then we can explore the callback success per segment. I suspect that recruiters discriminate more for certain kind of jobs. 